In [82]:
import pandas as pd
import nltk
import string
from sklearn import svm
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score



In [83]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [84]:
df = pd.read_csv("/content/tweets_suspect.csv") 

In [85]:
df

,message,label
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",1
1,is upset that he can't update his Facebook by ...,1
2,@Kenichan I dived many times for the ball. Man...,1
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",1
...,...,...
59995,"Really wants to go and see 17 again, because Z...",1
59996,@krissa22 Thank you!,1
59997,dreaming of you,1
59998,@TheEllenShow I saw a clip online! good show!,1


In [86]:
text = "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"

def tokenizer(text: str):
  text = text.lower()
  words = nltk.word_tokenize(text)
  return [word for word in words if word not in string.punctuation]

In [81]:
df['sentences_words'] = df["message"].apply(lambda x: tokenizer(x))
caractere = df['sentences_words']
cible = df['label']
x_train, x_test, y_train, y_test = train_test_split(caractere, cible, test_size=0.2,random_state=42)

In [87]:
df

,message,label
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",1
1,is upset that he can't update his Facebook by ...,1
2,@Kenichan I dived many times for the ball. Man...,1
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",1
...,...,...
59995,"Really wants to go and see 17 again, because Z...",1
59996,@krissa22 Thank you!,1
59997,dreaming of you,1
59998,@TheEllenShow I saw a clip online! good show!,1


In [88]:
model = Word2Vec(sentences=sentences_words, size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [89]:
def featureVecMethod(words, model, num_features):
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec, model[word])
    
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [90]:
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:  
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [91]:
trainDataVecs = getAvgFeatureVecs(x_train, model, 100)

<ipython-input-89-8eece7c34029>:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  featureVec = np.add(featureVec, model[word])


In [92]:
forest = RandomForestClassifier(n_estimators = 300)
forest = forest.fit(trainDataVecs, y_train)

In [93]:
testDataVecs = getAvgFeatureVecs(x_test, model, 100)

<ipython-input-89-8eece7c34029>:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  featureVec = np.add(featureVec, model[word])


In [94]:
predictions = forest.predict(testDataVecs)

In [95]:
precision, recall, fscore, _ =score(y_test, predictions, pos_label= 1, average='binary')

In [102]:
Accuracy = (predictions==y_test).sum() / len(predictions)

In [107]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision,3),
                                                         round(recall,3),
                                                         round((predictions==y_test).sum()/len(predictions),3)))

Precision: 0.9 / Recall: 1.0 / Accuracy: 0.9
